In [1]:
import pickle
import tensorflow
from keras.optimizers import SGD
from keras import Model
from keras.layers import (
    Input,
    TimeDistributed,
    LSTM,
    Dense,
    Input,
    Dense,
    Conv1D,
    Flatten, 
    Embedding)
from imblearn.over_sampling import SMOTE
import numpy

2022-07-15 17:47:30.687545: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-15 17:47:30.687569: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/aaron/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [17]:
HOME_DIR:str = '/home/aaron/timeseries_nlp'
constants_file = os.path.join(HOME_DIR, 'constants.py')
%run '/home/aaron/timeseries_nlp/constants.py'


In [3]:
with open(X_TRAIN_MULTI_INPUT_SAVE_FILE, "rb") as f:
        x_train = pickle.load(f)
with open(Y_TRAIN_MULTI_INPUT_SAVE_FILE, "rb") as f:
        y_train = pickle.load(f)
with open(X_TEST_MULTI_INPUT_SAVE_FILE, "rb") as f:
        x_test = pickle.load(f)
with open(Y_TEST_MULTI_INPUT_SAVE_FILE, "rb") as f:
        y_test = pickle.load(f)
with open(X_VAL_MULTI_INPUT_SAVE_FILE, "rb") as f:
        x_val = pickle.load(f)
with open(Y_VAL_MULTI_INPUT_SAVE_FILE, "rb") as f:
        y_val = pickle.load(f)
with open(EMBEDDING_MATRIX_SAVE_FILE, "rb") as f:
        embedding_matrix = pickle.load(f)

print("------Saving varaibles for reuse ------")
print(f"X_train shape: {x_train.shape}")
print(f"Y_train shape: {y_train.shape}")
print(f"X_test shape: {x_test.shape}")
print(f"Y_test shape: {y_test.shape}")
print(f"X_val shape: {x_val.shape}")
print(f"Y_val shape: {y_val.shape}")
print(f"Embedding shape: {embedding_matrix.shape}")
print(f"Total 0 values: {(y_train == 0).sum()}")
print(f"Total 1 values: {(y_train == 1).sum()}")

------Saving varaibles for reuse ------
X_train shape: (17212, 30, 200)
Y_train shape: (17212,)
X_test shape: (4782, 30, 200)
Y_test shape: (4782,)
X_val shape: (1913, 30, 200)
Y_val shape: (1913,)
Embedding shape: (10000, 300)
Total 0 values: 14308
Total 1 values: 2904


In [6]:
if BALANCE_DATA:
    arr = x_train.reshape(len(x_train), -1)
    sm = SMOTE(random_state=SEED)
    x_train_bal, y_train = sm.fit_resample(arr, y_train.ravel())
    print(f"After OverSampling, the shape of train_X: {x_train_bal.shape}")
    print(f"After OverSampling, the shape of train_y: {y_train.shape}")
    print(f"After OverSampling, counts of label '1': {sum(y_train == 1)}")
    print(f"After OverSampling, counts of label '0': {sum(y_train == 0)}")
    x_train = numpy.reshape(x_train_bal, (-1, x_train.shape[1], x_train.shape[2]))


(17212, 6000)
(17212,)
After OverSampling, the shape of train_X: (28616, 6000)
After OverSampling, the shape of train_y: (28616,)
After OverSampling, counts of label '1': 14308
After OverSampling, counts of label '0': 14308


In [7]:
embedding_layer = Embedding(
        MAX_VOCAB_SIZE,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False,)


In [8]:
def run_stacked_lstm():
    document_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
    embedding_sequences =embedding_layer(document_input)
    x = LSTM(12, return_sequences=True)(embedding_sequences)
    x = LSTM(12)(x)
    doc_model = Model(document_input, x)
    input_docs = Input(
                shape=(TIME_STEP, MAX_SEQUENCE_LENGTH), name="input_docs", dtype="int32"
            )

    x = TimeDistributed(doc_model, name="token_embedding_model")(input_docs)
    x = LSTM(12)(x)
    outputs = Dense(1, activation="sigmoid")(x)
    model = Model(input_docs, outputs)

    sgd = SGD(lr=LR, decay=1e-6, momentum=0.9, nesterov=True)

    model.compile(
                loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"]
            )
    model.summary()
    model.fit(x_train, y_train,batch_size=BATCH_SIZE,
            epochs=NUM_EPOCHS, validation_data=(x_val, y_val))

In [9]:
def run_lstm():
    document_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
    embedding_sequences =embedding_layer(document_input)
    x = LSTM(12)(embedding_sequences)
    doc_model = Model(document_input, x)
    input_docs = Input(
                shape=(TIME_STEP, MAX_SEQUENCE_LENGTH), name="input_docs", dtype="int32"
            )

    x = TimeDistributed(doc_model, name="token_embedding_model")(input_docs)
    x = LSTM(12)(x)
    outputs = Dense(1, activation="sigmoid")(x)
    model = Model(input_docs, outputs)

    sgd = SGD(lr=LR, decay=1e-6, momentum=0.9, nesterov=True)

    model.compile(
                loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"]
            )
    model.summary()
    model.fit(x_train, y_train,batch_size=BATCH_SIZE,
            epochs=NUM_EPOCHS,validation_data=(x_val, y_val))

In [10]:
def run_cnn():
    document_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
    embedding_sequences = embedding_layer(document_input)

    x = Conv1D(filters=300, kernel_size=5, padding="valid")(embedding_sequences)
    doc_model = Model(document_input, x)

    input_docs = Input(
            shape=(TIME_STEP, MAX_SEQUENCE_LENGTH), name="input_docs", dtype="int32"
        )

    x = TimeDistributed(doc_model, name="token_embedding_model")(input_docs)
    x = Conv1D(filters=300, kernel_size=5, padding="valid")(x)
    x = Flatten()(x)
    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(input_docs, outputs)

    opt = tensorflow.keras.optimizers.Adam(learning_rate=LR, beta_1=0.5, beta_2=0.999)
    model.compile(
                loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"]
            )
    model.summary()
    model.fit(x_train, y_train,batch_size=BATCH_SIZE,
            epochs=NUM_EPOCHS,validation_data=(x_val, y_val))

In [18]:
# run_cnn()
run_stacked_lstm()
# run_lstm()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_docs (InputLayer)     [(None, 30, 200)]         0         
                                                                 
 token_embedding_model (Time  (None, 30, 12)           3016224   
 Distributed)                                                    
                                                                 
 lstm_11 (LSTM)              (None, 12)                1200      
                                                                 
 dense_3 (Dense)             (None, 1)                 13        
                                                                 
Total params: 3,017,437
Trainable params: 17,437
Non-trainable params: 3,000,000
_________________________________________________________________
Epoch 1/5
224/224 [==============================] - 288s 1s/step - loss: 0.0000e+00 - accuracy: 0.5000 - val_

KeyboardInterrupt: 

In [72]:
with open(VOCAB_SAVE_FILE, "rb") as f:
        vocab = pickle.load(f)

norevisits = numpy.where(y_train == 0)
norevisits_indexes = norevisits[0][0:10]
revisits = numpy.where(y_train == 1)
revisits_indexes = revisits[0][0:10]
# print(result)

for index, value in enumerate(norevisits_indexes):
    print(f"Label {y_train[value]}")
    sample = x_train[value]
    EHR = sample[TIME_STEP - 1]
    for word in EHR:
        print(vocab[word], end = ' ')
        
for index, value in enumerate(revisits_indexes):
    print(f"Label {y_train[value]}")
    sample = x_train[value]
    EHR = sample[TIME_STEP - 1]
    for word in EHR:
        print(vocab[word], end = ' ')
        


Label 0
vegas bonsoir blk livecam [UNK] [UNK] openly reportedly [UNK] econo provides [UNK] [UNK] luther [UNK] [UNK] threw obey left [UNK] [UNK] [UNK] serving [UNK] relaxed [UNK] [UNK] idiots canoe [UNK] ctrl [UNK] [UNK] slate [UNK] [UNK] robust inline [UNK] wall backyard ntsc [UNK] instinct icelandic cbc [UNK] [UNK] molding gdp [UNK] [UNK] zinc tional [UNK] [UNK] patio tal wright oui astra acapulco [UNK] [UNK] pledge [UNK] lamborghini plantronics munich corrupted [UNK] [UNK] [UNK] [UNK] [UNK] lithuania [UNK] stroller whole account stellar [UNK] reed [UNK] [UNK] [UNK] bonjour pertinent whale [UNK] signing tin [UNK] gly largo cher                                                                                                         Label 0
nottinghamshire [UNK] [UNK] [UNK] echoes [UNK] sexy [UNK] [UNK] [UNK] [UNK] [UNK] decoder [UNK] monthly [UNK] [UNK] glazed obligated [UNK] [UNK] [UNK] accept ban node [UNK] [UNK] [UNK] homeland [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] solutions injunction 